基于


1.去掉变化最小的特征
说明：如果95%的实例的该特征的取值都是1，那就认为这个特征作用不大，如果100%都是1，该特征没意义。
应用条件：当特征是离散型变量时才可以使用该方法，对于连续型需要先离散化
优缺点：实际上不太会有95%以上都取某个值的特征存在，因此该方法简单但不好用，可以作为特征选择的预处理，先去掉那些取值变化小的特征，然后用其他特征选择方法进行进一步的特征选择。
例：去除有80%以上都是0或都是1的特征

In [3]:
from sklearn.feature_selection import VarianceThreshold
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

2.单变量特征选择
单变量特征选择能够对每一个特征进行测试，衡之间量该特征和响应变量之间的关系，根据得分去除特征。
2.1Pearson相关系数
该方法衡量的是变量之间的线性相关性，结果的取值范围为[-1,1],-1表示完全的负相关，+1表示完全的正相关，0表示没有线性相关。scipy的pearsonr方法能够计算相关系数和p-value

In [2]:
import numpy as np
from scipy.stats import  pearsonr

np.random.seed(0)
size=300
x=np.random.normal(0,1,size)
print("Lower noise",pearsonr(x,x+np.random.normal(0,1,size)))
print("Higher nois",pearsonr(x,x+np.random.normal(0,10,size)))

Lower noise (0.71824836862138408, 7.3240173129983507e-49)
Higher nois (0.057964292079338155, 0.31700993885324752)


说明：该例中比较了变量加入噪声前后的差异，当变量较小时，相关性强。
sklearn提供的f_regression方法能够批量计算特征的p-value。（sklearn的pipeline）
pearson相关系数的缺点：作为特征排序机制，只对线性关系敏感，如果关系是非线性的，即使两个变量具有一一对应的关系，pearson相关性也可能接近0.

注：如果仅仅根据相关系数这个值来判断的话，有时可能有很，最好将数据可视化进行检验。

2.2互信息和最大信息系数
用互信息直接进行特征选择的不便之处：1.它不属于度量方式，也没有办法归一化，在不同的数据集上的结果无法做比较；2.对于连续变量的计算不是很方便（X，y都是集合，都是离散的取值），通常变量需要先离散化，而互信息的结果对离散化的方式很敏感。
最大信息系数克服了这两个问题，它首先寻找一种最优的离散化方法，然后把互信息取值转换成一种度量方式，取值在[0,1]上。

from minepy import MINE

m = MINE()
x = np.random.uniform(-1, 1, 10000)
m.compute_score(x, x**2)
print( m.mic())
MIC的统计能力遭到了一些质疑，当零假设不成立时，MIC的统计就会受到影响。
在有的数据集上不存在这个问题，但有的数据集上就存在这个问题。


2.3 距离相关系数 (Distance correlation)
距离相关系数是为了克服Pearson相关系数的弱点而生的。在x和x^2这个例子中，即便Pearson相关系数是0，我们也不能断定这两个变量是独立的（有可能是非线性相关）；但如果距离相关系数是0，那么我们就可以说这两个变量是独立的。
R的energy包里提供了距离相关系数的实现，另外这是Python gist的实现。
#R-code
> x = runif (1000, -1, 1)
> dcor(x, x**2)
[1] 0.4943864

尽管有MIC和距离相关系数在了，但当变量之间的关系接近线性相关的时候，Pearson相关系数仍然是不可替代的。第一、Pearson相关系数计算速度快，这在处理大规模数据的时候很重要。第二、Pearson相关系数的取值区间是[-1，1]，而MIC和距离相关系数都是[0，1]。这个特点使得Pearson相关系数能够表征更丰富的关系，符号表示关系的正负，绝对值能够表示强度。当然，Pearson相关性有效的前提是两个变量的变化关系是单调的。

2.4 基于学习模型的特征排序 (Model based ranking)
这种方法的思路是直接使用你要用的机器学习算法，针对每个单独的特征和响应变量建立预测模型。其实Pearson相关系数等价于线性回归里的标准化回归系数。假如某个特征和响应变量之间的关系是非线性的，可以用基于树的方法（决策树、随机森林）、或者扩展的线性模型等。基于树的方法比较易于使用，因为他们对非线性关系的建模比较好，并且不需要太多的调试。但要注意过拟合问题，因此树的深度最好不要太大，再就是运用交叉验证。


In [1]:
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor

#Load boston housing dataset as an example
boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

rf = RandomForestRegressor(n_estimators=20, max_depth=4)
scores = []
for i in range(X.shape[1]):
     score = cross_val_score(rf, X[:, i:i+1], Y, scoring="r2",
                              cv=ShuffleSplit(len(X), 3, .3))
     scores.append((round(np.mean(score), 3), names[i]))
print (sorted(scores, reverse=True))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


NameError: name 'np' is not defined

3.线性模型和正则化
线性模型：用线性模型的系数来选择特征变量，重要的模型在特征中对应的系数比较大，在噪音不多的数据上，或者是数据量远远大于特征数的数据上，如果特征之间相对来说是比较独立的，那么即便是运用最简单的线性回归模型也一样能取得非常好的效果。

In [7]:
from sklearn.linear_model import  LinearRegression
import  numpy as np

np.random.seed(0)
size=5000
#A dataset with 3 features
x=np.random.normal(0,1,(size,3))
y=x[:,0]+2*x[:,1]+np.random.normal(0,2,size)
lr=LinearRegression()
lr.fit(x,y)

#A helper method for pretty-printing linear models
def pretty_print_linear(coefs,names=None,sort=False):
    if names==None:
        names=["x%s" % x for x in range(len(coefs))]
    lst=zip(coefs,names)
    if sort:
        lst=sorted(lst,key=lambda  x:-np.abs(x[0]))
    return "+".join("%s*%s" %(round(coef,3),name)
                    for coef, name in lst)
print("Linear model:",pretty_print_linear(lr.coef_))

Linear model: 0.984*x0+1.995*x1+-0.041*x2


3.1 L1正则化/Lasso
L1正则化将系数w的l1范数作为惩罚项加到损失函数上，由于正则项非零，这就迫使那些弱的特征所对应的系数变成0.因此L1正则化往往会使学到的模型很稀疏（系数w经常为0），这个特征使得L1正则化成为一种很好的特征选择方法。Skearn为线性回归提供了Lasso，为分类提供了L1逻辑回归。

In [9]:
from sklearn.linear_model import  Lasso
from sklearn.preprocessing import  StandardScaler
from  sklearn.datasets import  load_boston

boston=load_boston()
scaler=StandardScaler()
x=scaler.fit_transform(boston["data"])
y=boston["target"]
names=boston["feature_names"]
lasso=Lasso(alpha=.3)
lasso.fit(x,y)
print("Lasso model",pretty_print_linear(lasso.coef_,names,sort=True))

Lasso model -3.707*LSTAT+2.992*RM+-1.757*PTRATIO+-1.081*DIS+-0.7*NOX+0.631*B+0.54*CHAS+-0.236*CRIM+0.081*ZN+-0.0*INDUS+-0.0*AGE+0.0*RAD+-0.0*TAX


E:\Anaconda\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


3.3 L2正则化/Ridge regression
L2正则化将系数向量的L2范数添加到损失函数中，由于L2惩罚项中的系数是二次方的，这使得L2和L1有着诸多差异，最明显的一点就是L2正则化会让系数的取值变得平均。对于

In [10]:
from sklearn.linear_model import  Ridge
from sklearn.metrics import  r2_score
size=100
#We run the method 10 times with different random seeds
for i in range(10):
    print("Random seed %s" %i)
    np.random.seed(seed=i)
    x_seed=np.random.normal(0,1,size)
    x1=x_seed+np.random.normal(0,.1,size)
    x2=x_seed+np.random.normal(0,.1,size)
    x3=x_seed+np.random.normal(0,.1,size)
    y=x1+x2+x3+np.random.normal(0,1,size)
    x=np.array([x1,x2,x3]).T
    
    lr=LinearRegression()
    lr.fit(x,y)
    print("Linear model:",pretty_print_linear(lr.coef_))
    
    ridge=Ridge(alpha=10)
    ridge.fit(x,y)
    print("Ridge model", pretty_print_linear(ridge.coef_))

Random seed 0
Linear model: 0.728*x0+2.309*x1+-0.082*x2


Ridge model 0.938*x0+1.059*x1+0.877*x2
Random seed 1
Linear model: 1.152*x0+2.366*x1+-0.599*x2
Ridge model 0.984*x0+1.068*x1+0.759*x2
Random seed 2
Linear model: 0.697*x0+0.322*x1+2.086*x2
Ridge model 0.972*x0+0.943*x1+1.085*x2
Random seed 3
Linear model: 0.287*x0+1.254*x1+1.491*x2
Ridge model 0.919*x0+1.005*x1+1.033*x2
Random seed 4
Linear model: 0.187*x0+0.772*x1+2.189*x2
Ridge model 0.964*x0+0.982*x1+1.098*x2
Random seed 5
Linear model: -1.291*x0+1.591*x1+2.747*x2
Ridge model 0.758*x0+1.011*x1+1.139*x2
Random seed 6
Linear model: 1.199*x0+-0.031*x1+1.915*x2
Ridge model 1.016*x0+0.89*x1+1.091*x2
Random seed 7
Linear model: 1.474*x0+1.762*x1+-0.151*x2
Ridge model 1.018*x0+1.039*x1+0.901*x2
Random seed 8
Linear model: 0.084*x0+1.88*x1+1.107*x2
Ridge model 0.907*x0+1.071*x1+1.008*x2
Random seed 9
Linear model: 0.714*x0+0.776*x1+1.364*x2
Ridge model 0.896*x0+0.903*x1+0.98*x2


4.随机森林 
随机森林具有准确率高，鲁棒性好易于使用等优点，是目前最流行的机器学习算法之一。随机森林提供了两种特征选择方法：mean decrease imputity和mean decrease accuracy
4.1 平均不纯度减少（mean decr impurity）
随机森林由多个决策树构成。决策树中的每一个节点都是关于某个特征的条件，为的是将数据集按照不同的响应变量一分为二。利用不纯度可以确定节点（最优条件），对于分类问题，通常采用基尼不纯度或者信息增益，对于回归问题，通常采用的是方差或者最小二乘拟合。当训练决策树的时候，可以计算出每个特征减少了多少树的不纯度。对于一个决策树森林来说，可以算出每个特征平均减少了多少不纯度，并把它平均减少的不纯度作为特征选择的值。
下面的例子是sklearn中基于随机森林的特征重要度的度量方法：


In [11]:
from sklearn.datasets import  load_boston
from sklearn.ensemble import  RandomForestRegressor
import  numpy as np

#Load boston housing dataset as an example 
boston =load_boston()
x=boston["data"]
y=boston["target"]
names=boston["feature_names"]
rf=RandomForestRegressor()
rf.fit(x,y)
print("Features sorted by their scores:")
print(sorted(zip(map(lambda  x:round(x,4),rf.feature_importances_),names),reverse=True))

Features sorted by their scores:
[(0.4128, 'RM'), (0.38140000000000002, 'LSTAT'), (0.0877, 'DIS'), (0.031600000000000003, 'CRIM'), (0.021399999999999999, 'NOX'), (0.0212, 'PTRATIO'), (0.0117, 'AGE'), (0.010699999999999999, 'B'), (0.0097999999999999997, 'TAX'), (0.0071000000000000004, 'INDUS'), (0.0028, 'RAD'), (0.001, 'ZN'), (0.00080000000000000004, 'CHAS')]


In [23]:
size=1000
np.random.seed(seed=10)
x_seed=np.random.normal(0,1,size)
x0=x_seed+np.random.normal(0,.1,size)
x1=x_seed+np.random.normal(0,.1,size)
x2=x_seed+np.random.normal(0,.1,size)
x=np.array([x0,x1,x2]).T
y=x0+x1+x2

rf=RandomForestRegressor(n_estimators=20,max_features=2)
rf.fit(x,y)
print( "Scores for X0, X1, X2:", map(lambda x:round (x,3),
                                    rf.feature_importances_))

Scores for X0, X1, X2: <map object at 0x00000270EF5AFA58>


Scores for X0, X1, X2: [0.278, 0.66, 0.062]
当计算特征重要性时，可以看到X1的重要度比X2的重要度要高出10倍，但实际上他们真正的重要度是一样的。尽管数据量已经很大且没有噪音，且用了20棵树来做随机选择，但这个问题还是会存在。
需要注意的一点是，关联特征的打分存在不稳定的现象，这不仅仅是随机森林特有的，大多数基于模型的特征选择方法都存在这个问题。

In [34]:
from sklearn.cross_validation import  ShuffleSplit
from sklearn.metrics import  r2_score
from  collections import  defaultdict
x=boston["data"]
y=boston["target"]

rf=RandomForestRegressor()
scores=defaultdict(list)

#crossvalidate the scores on a number of different random splits of the data
for train_idx,test_idx in ShuffleSplit(len(X),100,.3):
    x_train,x_test=x[train_idx],x[test_idx]
    y_train,y_test=y[train_idx],y[test_idx]
    r=rf.fit(x_train,y_train)
    acc=r2_score(y_test,rf.predict(x_test))
    for i in range(x.shape[1]):
            x_t=x_test.copy()
            np.random.shuffle(x_t[:,i])
            shuff_acc=r2_score(y_test,rf.predict(x_t))
            scores[names[i]].append((acc-shuff_acc)/acc)
print("Feature sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for
            feat, score in scores.items()], reverse=True))

Feature sorted by their score:
[(0.7611, 'LSTAT'), (0.55979999999999996, 'RM'), (0.093899999999999997, 'DIS'), (0.041799999999999997, 'NOX'), (0.038899999999999997, 'CRIM'), (0.019400000000000001, 'PTRATIO'), (0.0154, 'TAX'), (0.0114, 'AGE'), (0.0054999999999999997, 'B'), (0.0051999999999999998, 'INDUS'), (0.0041000000000000003, 'RAD'), (0.00029999999999999997, 'ZN'), (0.00020000000000000001, 'CHAS')]


在这个例子当中，LSTAT和RM这两个特征对模型的性能有着很大的影响，打乱这两个特征的特征值使得模型的性能下降了73%和57%。注意，尽管这些我们是在所有特征上进行了训练得到了模型，然后才得到了每个特征的重要性测试，这并不意味着我们扔掉某个或者某些重要特征后模型的性能就一定会下降很多，因为即便某个特征删掉之后，其关联特征一样可以发挥作用，让模型性能基本上不变。

In [37]:
from sklearn.linear_model import  RandomizedLasso
from sklearn.datasets import load_boston
boston=load_boston()
#using the boston housing data
#Data gets scaled autoamtically by sklearn's implementation
x=boston["data"]
y=boston["target"]
names=boston["feature_names"]

rlasso=RandomizedLasso(alpha=0.025)
rlasso.fit(x,y)

print("Features sorted by their score:")
print(sorted(zip(map(lambda  x: round(x,4),rlasso.scores_),names),reverse=True))

Features sorted by their score:
[(1.0, 'RM'), (1.0, 'PTRATIO'), (1.0, 'LSTAT'), (0.62, 'B'), (0.59999999999999998, 'CHAS'), (0.40500000000000003, 'TAX'), (0.38, 'CRIM'), (0.22500000000000001, 'NOX'), (0.215, 'DIS'), (0.11, 'INDUS'), (0.044999999999999998, 'ZN'), (0.025000000000000001, 'RAD'), (0.02, 'AGE')]


在上边这个例子当中，最高的3个特征得分是1.0，这表示他们总会被选作有用的特征（当然，得分会收到正则化参数alpha的影响，但是sklearn的随机lasso能够自动选择最优的alpha）。接下来的几个特征得分就开始下降，但是下降的不是特别急剧，这跟纯lasso的方法和随机森林的结果不一样。能够看出稳定性选择对于克服过拟合和对数据理解来说都是有帮助的：总的来说，好的特征不会因为有相似的特征、关联特征而得分为0，这跟Lasso是不同的。对于特征选择任务，在许多数据集和环境下，稳定性选择往往是性能最好的方法之一。
5.2 递归特征消除 Recursive feature elimination (RFE)
递归特征消除的主要思想是反复的构建模型（如SVM或者回归模型）然后选出最好的（或者最差的）的特征（可以根据系数来选），把选出来的特征放到一遍，然后在剩余的特征上重复这个过程，直到所有特征都遍历了。这个过程中特征被消除的次序就是特征的排序。因此，这是一种寻找最优特征子集的贪心算法。
RFE的稳定性很大程度上取决于在迭代的时候底层用哪种模型。例如，假如RFE采用的普通的回归，没有经过正则化的回归是不稳定的，那么RFE就是不稳定的；假如采用的是Ridge，而用Ridge正则化的回归是稳定的，那么RFE就是稳定的。
Sklearn提供了RFE包，可以用于特征消除，还提供了RFECV，可以通过交叉验证来对的特征进行排序。

In [40]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

#use linear regression as the model
lr = LinearRegression()
#rank all features, i.e continue the elimination until the last one
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X,Y)

print("Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), names)))

Features sorted by their rank:
[(1, 'NOX'), (2, 'RM'), (3, 'CHAS'), (4, 'PTRATIO'), (5, 'DIS'), (6, 'LSTAT'), (7, 'RAD'), (8, 'CRIM'), (9, 'INDUS'), (10, 'ZN'), (11, 'TAX'), (12, 'B'), (13, 'AGE')]


In [10]:

import  numpy as np
import  math
math.log(78,2)
#np.linspace(1,10,10)


6.285402218862249